In [24]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../data'
group_dir_ = '../../data/groups/low'
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'

In [25]:
all_user = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
predictions = pd.read_pickle(os.path.join(dir_, 'post_filtering_old', str(0.005) + '_prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))

In [26]:
predictions[:5]

,uid,tid,rating
20,0,125,2.265003
32,0,177,2.514423
50,0,282,2.146869
61,0,350,2.423719
94,0,542,2.223415


In [27]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
7,0,3,0.25,1
11,0,7,2.50,3
32,0,18,1.50,2
34,0,20,2.50,3
45,0,31,5.00,17


In [28]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [29]:
num_user = len(predictions['uid'].unique())
num_user

220

In [7]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [8]:
# Count POP to DF
tid_list = []
pop_list = []
for i in all_user['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(all_user[all_user['tid']==i])/num_user*5)
    
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
166,166,3.840909
7209,7209,3.295455
350,350,3.113636
3761,3761,3.045455
80,80,3.022727
1017,1017,2.840909
1753,1753,2.750000
428,428,2.727273
200,200,2.681818
3159,3159,2.659091


In [9]:
top_percent = 0.01
long_tail = df_pop[int(len(df_pop)*top_percent):]
long_tail = long_tail['tid'].unique()
len(long_tail), len(df_pop), len(df_pop)-len(long_tail)

(54420, 54969, 549)

In [10]:
pop_4 = []
pop_20 = []
pop_50 = []

In [9]:
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_pops = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        pops =[]
        for j in (range(len(groups))):
            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    pops.append(pop_dict[tid])
        avg_pop = sum(pops)/len(pops)
        avg_pops.append(avg_pop)
        
    if top_n_size == 4:
        pop_4.append(avg_pops)
    elif top_n_size == 20:
        pop_20.append(avg_pops)
    elif top_n_size == 50:
        pop_50.append(avg_pops)
        
for i in pop_4:
    print(i)
for i in pop_20:
    print(i)
for i in pop_50:
    print(i)

In [30]:
top_n_size = 50
top_n_lists_g = []

for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 
        
        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

HitNOVs = []
for i in tqdm(range(len(groups_n))):
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]

    nov = 0
    for j in (range(len(groups))):

        group = groups[j]
        top_n_list = top_n_lists[j]

        for k in range(len(group)):
            for l in range(top_n_size):
                uid = group[k]
                tid = top_n_list[l]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                if len(t) > 0:
                    nov += (1-pop_dict[tid])
                    
    HitNOVs.append(nov/top_n_size/(len(groups)*len(groups[0])))
    
print(HitNOVs)


[0.0697799999999998, 0.0438411386593205, 0.03919019138755984, 0.03020139860139861, 0.027780303030303023, 0.028483766233766233, 0.020962662337662332]


In [76]:
top_n_size = 50
top_n_items_g = []
diversities = []
novelties = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_items = pd.DataFrame() 

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        if len(top_n_items) == 0:
            top_n_items = rating_table
        else:
            top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()
    top_n_items_g.append(top_n_items)   

diversity = []
for i in range(len(groups_n)):
    top_n_items = top_n_items_g[i]
    diversity.append(len(top_n_items)/len(all_user['tid'].unique()))
diversities.append(diversity)

novelty = []
for i in range(len(groups_n)):
    top_n_items = top_n_items_g[i]
    novelty.append(len(top_n_items[top_n_items['tid'].isin(long_tail)])/len(all_user['tid'].unique()))
novelties.append(novelty)

In [77]:
for diversity in diversities:
    print(diversity)
print()
for novelty in novelties:
    print(novelty)

[0.004602594189452237, 0.0038931033855445795, 0.0035656460914333535, 0.003274572941112263, 0.0031654205097418546, 0.0030562680783714456, 0.0030562680783714456]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [11]:
tid_pd = pd.DataFrame(data={'tid': all_user['tid'].unique()})
tid_pd = tid_pd.sort_values(by=['tid']).reset_index()
long_tail_list = tid_pd['tid'].isin(long_tail)
long_tail_list = list(long_tail_list)
long_tail_list.count(True)

54420

In [18]:
top_n_size = 50
top_n_lists_g = []
diversities = []
novelties = []

for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 
        
        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

diversity = []
novelty = []
for i in range(len(groups_n)):
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]


    hits = np.zeros(len(all_user['tid'].unique()), dtype=bool)
    for j in (range(len(groups))):

        group = groups[j]
        top_n_list = top_n_lists[j]

        for k in range(len(group)):
            for l in range(top_n_size):
                uid = group[k]
                tid = top_n_list[l]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                if len(t) > 0:
                    hits[int(tid)] = True

    diversity.append(list(hits).count(True)/len(all_user['tid'].unique()))
    novelty.append(list(hits&long_tail_list).count(True)/len(all_user['tid'].unique()))
diversities.append(diversity)
novelties.append(novelty)
    
for diversity in diversities:
    print(diversity)
print()
for novelty in novelties:
    print(novelty)


[0.0037475668103840345, 0.0026014662809947424, 0.0023649693463588567, 0.001710054758136404, 0.0015099419672906548, 0.0016554785424511998, 0.0012188688169695646]

[0.0004729938692717714, 0.0001819207189506813, 0.00014553657516054504, 0.00014553657516054504, 5.457621568520439e-05, 0.00021830486274081756, 0.00016372864705561317]
